In [ ]:
from hedge_stand import HedgeST,HedgeSTBoll
from datetime import date, datetime
from demeter import TokenInfo, PoolBaseInfo, Actuator, Strategy, Asset, AccountStatus, BuyAction, SellAction, RowData, \
    ChainType
import optunity
import optunity.metrics
from decimal import Decimal
import pandas as pd
import  matplotlib.pylab as plt
# import logging 
# from logging import handlers
from load_data import pool_id_1_eth_u_500,pool_id_1_wbtc_u_3000,pool_id_1_wbtc_u_500

from strategy_ploter import  plot_position_return_decomposition

import quantstats as qs
qs.extend_pandas()

%matplotlib inline

def backtest_boll(a, hedge_spread_split,hedge_spread_rate,alpha,boll_period_n):
    # global RUNNING_TIME
    # print(f"==================spread running time {RUNNING_TIME}==================")

    decimal_a = Decimal(a).quantize(Decimal('0.00'))
    decimal_hedge_spread_split = Decimal(hedge_spread_split).quantize(Decimal('0.0'))
    decimal_hedge_spread_rate = Decimal(hedge_spread_rate).quantize(Decimal('0.00'))

    ema_max_spread_rate = round(0.2,4)



    #收益计算基础参数
    # net_value_base = 'ETH'

    runner_instance = Actuator(PAIR_POOL)
    # runner_instance.enable_notify = False
    runner_instance.strategy = HedgeSTBoll(a=decimal_a,hedge_spread_split = decimal_hedge_spread_split,hedge_spread_rate=decimal_hedge_spread_rate,alpha=alpha,ema_max_spread_rate=ema_max_spread_rate,period_n=int(boll_period_n))
    runner_instance.set_assets([Asset(USDC, 1000000)])
    save_path = f"../demeter/data/ETH/{POOL_ID}"
    runner_instance.data_path = save_path
    runner_instance.load_data(ChainType.Ethereum.name,
                                POOL_ID,
                                DATE_START,
                               DATE_END)
    runner_instance.run(enable_notify=False)

    hedge_count = runner_instance.strategy.hedge_count
    outside_ema_count = runner_instance.strategy.outside_ema_count

    # df_status = pd.DataFrame(runner_instance.account_status_list)

    total_net_value = runner_instance.final_status.net_value
    
    final_total_usdc_value = round(total_net_value + runner_instance.strategy.e.df['total'].iloc[-1],3)
    
    final_price = runner_instance.final_status.price

    final_total_eth_value = round(final_total_usdc_value / final_price,3)


    notice = f"ema+alpha:{RUNNING_TIME} times, a:{decimal_a}, hedge_spread_split:{decimal_hedge_spread_split}, hedge_spread_rate:{decimal_hedge_spread_rate},alpa:{alpha}"
    result =f" result: outside_ema_count:{outside_ema_count},hedge count:{hedge_count} final_total_eth_value:{final_total_eth_value},final_total_usdc_value:{final_total_usdc_value}"  
    print(notice)
    print(result)
    # if SEND_NOTICE:
    #     send_notice('CEX_Notify',notice)

    # RUNNING_TIME +=1

    # if NET_VALUE_BASE == 'USDC':
    #     print(final_total_usdc_value)
    #     return float(final_total_usdc_value)
    #     # profit_rate_usdc = profit_usdc / runner_instance.strategy.init_total_usdc
    # else:
    #     print(float(final_total_usdc_value / final_price))
    #     return float(final_total_usdc_value / final_price)


    return runner_instance


def backtest_instance(a, hedge_spread_split,hedge_spread_rate,alpha):
    # global RUNNING_TIME
    # print(f"==================spread running time {RUNNING_TIME}==================")

    decimal_a = Decimal(a).quantize(Decimal('0.00'))
    decimal_hedge_spread_split = Decimal(hedge_spread_split).quantize(Decimal('0.0'))
    decimal_hedge_spread_rate = Decimal(hedge_spread_rate).quantize(Decimal('0.00'))

    alpha = round(alpha,4)
    # ema_max_spread_rate = round(ema_max_spread_rate,4)

    eth = TokenInfo(name="eth", decimal=18)
    usdc = TokenInfo(name="usdc", decimal=6)
    pool = PoolBaseInfo(usdc, eth, 0.05, usdc)

    #收益计算基础参数
    # net_value_base = 'ETH'

    runner_instance = Actuator(pool)
    # runner_instance.enable_notify = False
    runner_instance.strategy = HedgeST(decimal_a,decimal_hedge_spread_split,decimal_hedge_spread_rate,alpha)
    runner_instance.set_assets([Asset(usdc, 1000000)])
    save_path = f"../demeter/data/ETH/{pool_id_1_eth_u_500}"
    runner_instance.data_path = save_path
    runner_instance.load_data(ChainType.Ethereum.name,
                                pool_id_1_eth_u_500,
                                DATE_START,
                               DATE_END)
    runner_instance.run(enable_notify=False)

    hedge_count = runner_instance.strategy.hedge_count
    outside_ema_count = runner_instance.strategy.outside_ema_count

    # df_status = pd.DataFrame(runner_instance.account_status_list)

    total_net_value = runner_instance.final_status.net_value
    
    final_total_usdc_value = round(total_net_value + runner_instance.strategy.e.df['total'].iloc[-1],3)
    
    final_price = runner_instance.final_status.price

    final_total_eth_value = round(final_total_usdc_value / final_price,3)

    #处理df_status
    df_status = pd.DataFrame(runner_instance.account_status_list)
    df_merge = pd.merge(e.df, df_status, on='timestamp', how='inner')

    df_merge['total_value'] = df_merge['total'] + df_merge['net_value']
    df_merge['total_value_eth'] = df_merge['total_value'] / df_merge['price']

    df_merge.set_index('timestamp', inplace=True)




    notice = f"ema+alpha:{RUNNING_TIME} times, a:{decimal_a}, hedge_spread_split:{decimal_hedge_spread_split}, hedge_spread_rate:{decimal_hedge_spread_rate},alpa:{alpha}"
    result =f" result: outside_ema_count:{outside_ema_count},hedge count:{hedge_count} final_total_eth_value:{final_total_eth_value},final_total_usdc_value:{final_total_usdc_value}"  
    print(notice)
    print(result)
    # if SEND_NOTICE:
    #     send_notice('CEX_Notify',notice)

    # RUNNING_TIME +=1

    # if NET_VALUE_BASE == 'USDC':
    #     print(final_total_usdc_value)
    #     return float(final_total_usdc_value)
    #     # profit_rate_usdc = profit_usdc / runner_instance.strategy.init_total_usdc
    # else:
    #     print(float(final_total_usdc_value / final_price))
    #     return float(final_total_usdc_value / final_price)


    return runner_instance





In [ ]:

def generate_dfmerge(runner_instance):
    e = runner_instance.strategy.e

    total_net_value = runner_instance.final_status.net_value
        
    final_total_usdc_value = total_net_value + runner_instance.strategy.e.df['total'].iloc[-1]


    final_price = runner_instance.final_status.price
    # if net_value_base == 'USDC':
    profit_usdc = round(final_total_usdc_value- runner_instance.strategy.init_total_usdc,3)
    profit_rate_usdc = round(profit_usdc / runner_instance.strategy.init_total_usdc,3)
    # else:
    profit_eth = round(final_total_usdc_value/final_price - runner_instance.strategy.init_total_symbol,3)
    profit_rate_eth = round(profit_eth / runner_instance.strategy.init_total_symbol,3)

    print(f"USDC profit: {profit_usdc}, profit_rate: {profit_rate_usdc}")

    print(f"ETH profit: {profit_eth}, profit_rate: {profit_rate_eth}")

    df_status = pd.DataFrame(runner_instance.account_status_list)
    df_merge = pd.merge(e.df, df_status, on='timestamp', how='inner')

    df_merge['total_value'] = df_merge['total'] + df_merge['net_value']

    df_merge.set_index('timestamp', inplace=True)

    df_merge['total_value_profit_rate'] = df_merge['total_value'] / runner_instance.strategy.init_total_usdc


    df_merge['total_value_eth'] = df_merge['total_value'] / df_merge['price']

    df_merge['net_value_eth'] = df_merge['net_value'] / df_merge['price']

    df_merge['total_value_eth_profit_rate'] = df_merge['total_value_eth'] / runner_instance.strategy.init_total_symbol

    df_merge_f = df_merge.apply(pd.to_numeric, downcast='float')
   
    df_merge_f['net_value_rate'] = df_merge_f['net_value'] / df_merge_f['net_value'].iloc[0]

    df_merge_f['price_rate'] = df_merge_f['price'] / df_merge_f['price'].iloc[0]
    return df_merge_f

In [ ]:
# pd.options.plotting.backend = "plotly"

CHAIN_ID = 'ETH'
NET_VALUE_BASE = 'WBTC'
RUNNING_TIME = 1
SEND_NOTICE = False
str_date_start = '2022-9-1'
str_date_end = '2022-11-30'
POOL_ID = pool_id_1_wbtc_u_3000
#eth pool
# eth = TokenInfo(name='eth', decimal=18)
# USDC = TokenInfo(name="usdc", decimal=6)
# PAIR_POOL = PoolBaseInfo(USDC, eth, 0.05, USDC)

# wbtc pool
wbtc = TokenInfo(name='wbtc', decimal=8)
USDC = TokenInfo(name="usdc", decimal=6)
PAIR_POOL = PoolBaseInfo(wbtc, USDC, 0.3, USDC)

DATE_START = datetime.strptime(str_date_start, "%Y-%m-%d").date()

DATE_END = datetime.strptime(str_date_end, "%Y-%m-%d").date()

# A1 test

In [5]:
a = 1.15
hedge_spread_split = 2.7
hedge_spread_rate = 0.9
alpha = 0.037
ema_max_spread_rate=0.2
period_n = 19

instance=backtest_boll(a=a,hedge_spread_split=hedge_spread_split,hedge_spread_rate=hedge_spread_rate,alpha=alpha,boll_period_n=period_n)

df_merge = generate_dfmerge(instance)


filename = f"{NET_VALUE_BASE}_{DATE_START}_{DATE_END}_{a}_{hedge_spread_split}_{hedge_spread_rate}_{alpha}_{period_n}_{POOL_ID}.html"
# df_merge['total_value'].plot_snapshot()
qs.reports.html(df_merge['total_value_eth'],benchmark=df_merge['net_value_eth'],mode='full',periods_per_year=365, download_filename=f"./reports/{filename}",output='report')

# fig = df_merge[['price_rate','total_value_eth_profit_rate','total_value_profit_rate']].plot().get_figure()

# fig.savefig('./fig/test.pdf')

# plot = df_merge[['price_rate','total_value_eth_profit_rate','net_value_rate','total_value_profit_rate']].plot()

# plot.write_image(f"./fig/{NET_VALUE_BASE}_{DATE_START}_{DATE_END}_{a}_{hedge_spread_split}_{hedge_spread_rate}_{alpha}_{period_n}.png")

# plot.show()

2023-01-10 14:01:51,931 - INFO - start load files from 2022-06-01 to 2022-12-30...


init parameters: a:1.15, hedge_spread_split:2.7, hedge_spread_rate:0.90,alpa:0.037,period_n:19



2023-01-10 14:01:59,273 - INFO - load file complete, preparing...
2023-01-10 14:02:08,793 - INFO - data has benn prepared
2023-01-10 14:02:08,794 - INFO - init strategy...


add ema column with alpha 0.037
306718


2023-01-10 14:02:11,419 - INFO - start main loop...


...

 46%|███████████████████████████████████████████████▎                                                       | 140726/306718 [41:29<1:02:53, 43.99it/s]

2022-09-06 17:22:00 hedge lower sell ETH,c_amount:27.58515170128224166502785375,f_amount:21.66979420890715925587004586, upper:22555.28,middle:21101.01,lower,19646.74, trade price: 19139.46042220326610199160088,ema:19505.589361317026,trade amount: 5.221964299207242216391872310, hedge count:6


 46%|███████████████████████████████████████████████▎                                                       | 140893/306718 [41:33<1:06:44, 41.41it/s]

2022-09-06 20:10:00 hedge lower sell ETH,c_amount:32.70810403243329109006796975,f_amount:16.44782990969991703947817355, upper:22555.28,middle:21101.01,lower,19646.74, trade price: 18766.12118345867998616314220,ema:18907.59806819193,trade amount: 5.221964299207242216391872310, hedge count:7


 46%|███████████████████████████████████████████████▊                                                       | 142300/306718 [42:34<3:36:05, 12.68it/s]

In [ ]:
# qs.plots.snapshot(df_merge['total_value_eth'])

# A2 test

In [ ]:
a = 1.15
POOL_ID = pool_id_1_wbtc_u_500
PAIR_POOL = PoolBaseInfo(wbtc, USDC, 0.05, USDC)

hedge_spread_split = 2.7
hedge_spread_rate = 0.9
alpha = 0.037
ema_max_spread_rate=0.2
period_n2 = 19

instance2=backtest_boll(a=a,hedge_spread_split=hedge_spread_split,hedge_spread_rate=hedge_spread_rate,alpha=alpha,boll_period_n=period_n2)

df_merge2 = generate_dfmerge(instance2)

# fig = df_merge[['price_rate','total_value_eth_profit_rate','total_value_profit_rate']].plot().get_figure()

# fig.savefig('./fig/test.pdf')

# plot2 = df_merge2[['price_rate','total_value_eth_profit_rate','net_value_rate','total_value_profit_rate']].plot()

# plot2.write_image(f"./fig/{NET_VALUE_BASE}_{DATE_START}_{DATE_END}_{a}_{hedge_spread_split}_{hedge_spread_rate}_{alpha}_{period_n2}.png")

# plot2.show()
filename = f"{NET_VALUE_BASE}_{DATE_START}_{DATE_END}_{a}_{hedge_spread_split}_{hedge_spread_rate}_{alpha}_{period_n2}_{POOL_ID}.html"
# df_merge['total_value'].plot_snapshot()
qs.reports.html(df_merge2['total_value_eth'],benchmark=df_merge2['net_value_eth'],mode='full',periods_per_year=365, download_filename=f"./reports/{filename}",output='report')

In [ ]:
# qs.reports.html(df_merge2['total_value_eth'],benchmark=df_merge2['net_value_eth'],mode='full',periods_per_year=365, download_filename=f"./reports/{filename}",output='report')

# a3 ema hedge

In [ ]:
a = 1.20
hedge_spread_split = 2.5
hedge_spread_rate = 0.86
alpha3 = 0.037
ema_max_spread_rate2=0.2

instance3=backtest_instance(a,hedge_spread_split,hedge_spread_rate,alpha3)

df_merge3 = generate_dfmerge(instance3)

filename = f"{NET_VALUE_BASE}_{DATE_START}_{DATE_END}_{a}_{hedge_spread_split}_{hedge_spread_rate}_{alpha}.html"
# df_merge['total_value'].plot_snapshot()
qs.reports.html(df_merge['total_value'],benchmark=df_merge['price'],mode='full',periods_per_year=365, download_filename=f"./reports/{filename}",output='report')

# plot3 = df_merge3[['price_rate','total_value_eth_profit_rate','net_value_rate','total_value_profit_rate']].plot()

# plot3.write_image(f"./fig/{NET_VALUE_BASE}_{DATE_START}_{DATE_END}_{a}_{hedge_spread_split}_{hedge_spread_rate}_{alpha3}.png")

# plot3.show()
filename = f"{NET_VALUE_BASE}_{DATE_START}_{DATE_END}_{a}_{hedge_spread_split}_{hedge_spread_rate}_{alpha3}.html"
# df_merge['total_value'].plot_snapshot()
qs.reports.html(df_merge3['total_value_eth'],benchmark=df_merge3['net_value_eth'],mode='full',periods_per_year=365, download_filename=f"./reports/{filename}",output='report')